In [11]:
#!/usr/bin/env python
"""
Using Keras to implement a 1D convolutional neural network (CNN) for timeseries prediction.
"""

from __future__ import print_function, division

import numpy as np
from keras.layers import Input, Convolution1D, Dense, MaxPooling1D, Flatten, LeakyReLU
from keras.models import Model
from keras.models import Sequential
import pandas as pd


__date__ = '2018-08-07'


def make_timeseries_regressor(window_size, kernel_size, nb_input_series=1, nb_outputs=1, filters=4):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.

    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).

    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int kernel_size: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `kernel_size`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int filters: The number of different filters to learn (roughly, input patterns to recognize).
    """
    
    # The first conv layer learns `filters` filters (aka kernels), each of size ``(kernel_size, nb_input_series)``.
    # Its output will have shape (None, window_size - kernel_size + 1, filters), i.e., for each position in
    # the input timeseries, the activation of each filter at that position.
        
    inputs = Input(shape=(window_size, nb_input_series))
    x = Convolution1D(filters, 1, padding='causal', name='initial_conv')(inputs)
    x = Convolution1D(activation='relu', filters=filters, kernel_size=kernel_size, padding='causal', dilation_rate=1)(x)
    #x = MaxPooling1D()(x) # Downsample the output of convolution by 2X.
    x = Convolution1D(activation='relu', filters=filters, kernel_size=kernel_size, padding='causal', dilation_rate=2)(x)
    #x = MaxPooling1D()(x)
    x = Convolution1D(activation='relu', filters=filters, kernel_size=kernel_size, padding='causal', dilation_rate=4)(x)
    x = Flatten()(x)
    preds = Dense(nb_outputs, activation='linear')(x)
    
    model = Model(inputs=inputs, outputs=preds)

    model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mse'])
    # To perform (binary) classification instead:
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model


def make_timeseries_instances(timeseries, window_size):
    """Make input features and prediction targets from a `timeseries` for use in machine learning.

    :return: A tuple of `(X, y, q)`.  `X` are the inputs to a predictor, a 3D ndarray with shape
      ``(timeseries.shape[0] - window_size, window_size, timeseries.shape[1] or 1)``.  For each row of `X`, the
      corresponding row of `y` is the next value in the timeseries.  The `q` or query is the last instance, what you would use
      to predict a hypothetical next (unprovided) value in the `timeseries`.
    :param ndarray timeseries: Either a simple vector, or a matrix of shape ``(timestep, series_num)``, i.e., time is axis 0 (the
      row) and the series is axis 1 (the column).
    :param int window_size: The number of samples to use as input prediction features (also called the lag or lookback).
    """
    timeseries = np.asarray(timeseries)
    assert 0 < window_size < timeseries.shape[0]
    
    X = []
    y = []
    q = []
    
    for start in range(0, timeseries.shape[0] - window_size):
        X.append(np.array(timeseries[start:start + window_size]))
        y.append(np.array(timeseries[start + window_size]))
        q.append([timeseries[-(start + window_size):]])
        
    X = np.atleast_3d(X)
    y = np.asarray(y)
    q = np.atleast_3d(q)
        
    print("X", X.shape)
#     print(X)
#     print()
    
    print("y", y.shape)
#     print(y)
#     print()
    
#     X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
#     y = timeseries[window_size:]
#     print('y', y.shape, y)
    q = np.atleast_3d([timeseries[-window_size:]])
    return X, y, q


def evaluate_timeseries(timeseries, window_size):
    """Create a 1D CNN regressor to predict the next value in a `timeseries` using the preceding `window_size` elements
    as input features and evaluate its performance.

    :param ndarray timeseries: Timeseries data with time increasing down the rows (the leading dimension/axis).
    :param int window_size: The number of previous timeseries values to use to predict the next.
    """
    kernel_size = 5
    filters = 4
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T       # Convert 1D vectors to 2D column vectors

    nb_samples, nb_series = timeseries.shape
    print('\n\nTimeseries ({} samples by {} series):\n'.format(nb_samples, nb_series), timeseries)
    model = make_timeseries_regressor(window_size=window_size, kernel_size=kernel_size, nb_input_series=nb_series, nb_outputs=nb_series, filters=filters)
    print('\n\nModel with input size {}, output size {}, {} conv filters of length {}'.format(model.input_shape, model.output_shape, filters, kernel_size))
    model.summary()

    X, y, q = make_timeseries_instances(timeseries, window_size)
    
    #epochs = int(np.sqrt(X.shape[0] * X.shape[1]))
    epochs = X.shape[0] * X.shape[1]
    print("epochs ", epochs)
    
    print('\n\nInput features:', X, '\n\nOutput labels:', y, '\n\nQuery vector:', q, sep='\n')
    test_size = int(0.2 * nb_samples)           
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    
    model.fit(X_train, y_train, epochs=epochs, batch_size=256, verbose=False)


#     pred = model.predict(X_test)
#     print('\n\nactual', 'predicted', sep='\t')
#     for actual, predicted in zip(y_test, pred.squeeze()):
#         print(actual.squeeze(), predicted, sep='\t')
#     print('next', model.predict(q).squeeze(), sep='\t')
    
    print()
    score = model.evaluate(X_test, y_test, verbose=0)
    print('(loss, mean_absolute_error, mean_squared_error) = ', score)


def play():
    """Prepare input data, build model, evaluate."""
    np.set_printoptions(threshold=25)
    ts_length = 1000
    window_size = 50

    print('\nSimple single timeseries vector prediction')
    timeseries = np.arange(ts_length)                   # The timeseries f(t) = t
    evaluate_timeseries(timeseries, window_size)

    print('\nMultiple-input, multiple-output prediction')
    timeseries = np.array([np.arange(ts_length), -np.arange(ts_length)]).T      # The timeseries f(t) = [t, -t]
    evaluate_timeseries(timeseries, window_size)




In [2]:
def load_timeseries(snap_fname):
    time_series_len = 128
    timeseries = []
    with open(snap_fname, 'r') as fin:
        for l in fin:
            tokens = l.strip().split('\t')
            if len(tokens) != time_series_len:
                continue
            
            timeseries.append([float(x) for x in tokens])
    # Transposing to make time = the rows (axis=0)
    timeseries = np.asarray(timeseries).T
    return timeseries

In [3]:
def load_m4_timeseries(fname):
    data = pd.read_csv(fname)
    data = data.fillna(0)
    data = data.drop(columns=['V1']).values.T
    return data

In [4]:
#timeseries = load_timeseries("MemePhr.txt")
fname = '/Users/bluebalam/projectsX/libreai/projects/minerva_experimental/minerva/minerva/M4_benchmark/M4-methods/Dataset/Train/Quarterly-train.csv'
timeseries = load_m4_timeseries(fname)

In [5]:
timeseries = timeseries[:,0:15]

In [6]:
timeseries.shape

(866, 15)

In [7]:
timeseries[:10,:4]

array([[7407.41231382, 7552.45461911, 8463.8421932 , 8498.94119397],
       [7528.5660743 , 7541.77457066, 8366.10230911, 8409.92644199],
       [7374.70922497, 7466.56833608, 8269.50219151, 8391.44138144],
       [7395.51484763, 7550.33335403, 8256.98532453, 8292.86031049],
       [7654.00798853, 8067.13152222, 8726.91764696, 8798.52111766],
       [7686.84783506, 8063.70101739, 8733.24359072, 8753.99035458],
       [7578.19074266, 7901.02931239, 8664.26008696, 8740.06255558],
       [7904.37671607, 8155.38731599, 8717.39456788, 8695.54065076],
       [7744.04925353, 8031.01032808, 8662.13972695, 8627.44748783],
       [7889.90901325, 8023.24000549, 8629.10189569, 8525.99342353]])

In [13]:
window_size = 4
#make_timeseries_instances(timeseries[:16,:4], window_size)
#evaluate_timeseries(timeseries[:16,:4], window_size)
evaluate_timeseries(timeseries, window_size)



Timeseries (866 samples by 15 series):
 [[7407.41231382 7552.45461911 8463.8421932  ...  926.9
  5370.         1527.        ]
 [7528.5660743  7541.77457066 8366.10230911 ...  953.5
  4940.         1576.        ]
 [7374.70922497 7466.56833608 8269.50219151 ...  945.1
  5660.         1641.        ]
 ...
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 [   0.            0.            0.         ...    0.
     0.            0.        ]]


Model with input size (None, 4, 15), output size (None, 15), 4 conv filters of length 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 4, 15)             0         
_________________________________________________________________
initial_conv (Conv1D)        (None, 4, 4)              64        
___________________